In [ ]:
#hide   
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
from pprint import pprint as pp
from datetime import datetime as dt
from typing import *
import rfpy
from rfpy.constants import *
from rfpy.parser import *
from rfpy.utils import *
from rfpy.blocks import *
from rfpy.main import export_metadata
from nbdev.showdoc import *
from fastcore.xtras import Path
from fastcore.foundation import L
from fastcore.test import *
import numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
entrada = Path(r'C:\Users\rsilva\Downloads\entrada')
saida = Path(r'C:\Users\rsilva\Downloads\saida')
csv1 = saida / 'pmec' / 'rfeye002304_SLMA_PEAK_200828_132002_Bloco_63_ThreadId_80_20200828-132101_20200831-031400.csv'
csv2 = saida / '201201_T153421_OneThreadID_Bloco_63_ThreadId_110_20201201-153421_20201201-161353.csv'

In [ ]:
arquivos = get_files(entrada) ;  arquivos

(#3) [Path('C:/Users/rsilva/Downloads/entrada/201201_T153421_OneThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/201201_T154509_MultiplesThreadID.bin'),Path('C:/Users/rsilva/Downloads/entrada/pmec/rfeye002304_SLMA_PEAK_200828_132002.bin')]

In [ ]:
file1 = arquivos[-1]
file2 = arquivos[0]
file3 = arquivos[1]

In [ ]:
%%time
one = parse_bin(file2)

Wall time: 1.53 s


In [ ]:
one

{'file_version': 22,
 'string': 'CRFS DATA FILE V022',
 'blocks': defaultdict(fastcore.foundation.L,
             {(21,
               0): (#1) [((36, 123), <rfpy.blocks.DType21 object at 0x000002B84A1DB408>)],
              (41,
               0): (#2) [((128, 203), <rfpy.blocks.DType41 object at 0x000002B84A1DB0C8>),((208, 335), <rfpy.blocks.DType41 object at 0x000002B84C413248>)],
              (24,
               5): (#1) [((340, 375), <rfpy.blocks.DType24 object at 0x000002B84139A808>)],
              (24,
               110): (#1) [((380, 427), <rfpy.blocks.DType24 object at 0x000002B82E9BE388>)],
              (63,
               110): (#46382) [((432, 2583), <rfpy.blocks.DType63 object at 0x000002B84A1D4E48>),((2588, 4739), <rfpy.blocks.DType63 object at 0x000002B84A1D4588>),((4744, 6895), <rfpy.blocks.DType63 object at 0x000002B84A1D4C08>),((6900, 9051), <rfpy.blocks.DType63 object at 0x000002B84A1D4548>),((9056, 11207), <rfpy.blocks.DType63 object at 0x000002B84A1D4988>),((11

In [ ]:
%%time
blocks = L(*[v for k,v in one['blocks'].items() if k[0] in SPECTRAL_BLOCKS])
levels = extract_block_levels(blocks)

Wall time: 58.2 s


In [ ]:
b = blocks[0][1]

In [ ]:
start = BYTES_63[21].stop + 8 + b.n_tunning * 4 + b.n_agc 
stop = start + b.ndata

In [ ]:
start, stop

(95, 2143)

In [ ]:
%%time
df = pd.read_csv(csv2).set_index('wallclock_datetime')

Wall time: 202 ms


In [ ]:
subset = df.iloc[0:10000]
start = subset.start_byte[0]
stop = subset.stop_byte[-1]
one = parse_bin(file2, slice_=slice(start,stop))
blocks = L(*[v for k,v in one['blocks'].items() if k[0] in SPECTRAL_BLOCKS])
levels = extract_block_levels(blocks)

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 2048 and the array at index 9999 has size 0

In [ ]:
df.head()

,start_byte,stop_byte,bw,data_size,gerror,gflags,group_id,id_antenna,level_offset,n_agc,...,start_mega,start_mili,step,stop_channel,stop_mega,stop_mili,thread_id,type,unit,file_version
wallclock_datetime,,,,,,,,,,,,,,,,,,,,,
2020-12-01 15:34:21.578869,432,2583,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,110,63,%,22
2020-12-01 15:34:21.714679,2588,4739,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,110,63,%,22
2020-12-01 15:34:21.920160,4744,6895,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,110,63,%,22
2020-12-01 15:34:22.134015,6900,9051,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,110,63,%,22
2020-12-01 15:34:22.346173,9056,11207,40,2136,-1,-1,0,0,-20,7,...,70,0,0.019541,0,110,0,110,63,%,22
